In [43]:
language = 'pt'

# 1. Gravação de Áudio Com Python (e Uma Pitada de JavaScript) 🎤

In [44]:
# Referência do código Javascript para gravação do áudio: https://gist.github.com/korakot/c21c3476c024ad6d56d5f48b0bca92be
from time import sleep
from IPython.display import Audio, display, Javascript
from google.colab import output
from base64 import b64decode

# Código JavaScript para gravar áudio do usuário usando a "MediaStream Recording API"
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  # Executa o código JavaScript para gravar o áudio
  display(Javascript(RECORD))
  # Recebe o áudio gravado como resultado do JavaScript
  js_result = output.eval_js('record(%s)' % (sec * 1000))
   # Decodifica o áudio em base64
  audio = b64decode(js_result.split(',')[1])
  # Salva o áudio em um arquivo
  file_name = 'request_audio.wav'
  with open(file_name, 'wb') as f:
    f.write(audio)
  # Retorna o caminho do arquivo de áudio (pasta padrão do Google Colab)
  return f'/content/{file_name}'
print("Esta é uma aplicação de IA que irá te auxiliar a traduzir textos, expressões e palavras de Português para Inglês, trazendo algumas aplicações auxiliar no entendimento da língua Inglesa.")
sleep(3)
print('\n'*3)

# Grava o áudio do usuário por um tempo determinado (padrão 5 segundos)
print('Ouvindo...\n')
record_file = record()

# Exibe o áudio gravado
display(Audio(record_file, autoplay=False))

Esta é uma aplicação de IA que irá te auxiliar a traduzir textos, expressões e palavras de Português para Inglês, trazendo algumas aplicações auxiliar no entendimento da língua Inglesa.




Ouvindo...



<IPython.core.display.Javascript object>

# 2. Reconhecimento de Fala com Whisper (OpenAI) 🧠

In [45]:
!pip install git+https://github.com/openai/whisper.git -q

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [46]:
import whisper

# Selecione o modelo do Whisper que melhor atenda às suas necessidades:
# https://github.com/openai/whisper#available-models-and-languages
model = whisper.load_model("small")

# Transcreve o audio gravado anteriormente.
result = model.transcribe(record_file, fp16=False, language=language)
transcription = result["text"]
print(transcription)

 Você pode cuidar desta criança?


# 3. Integração com a API do ChatGPT 💬

In [47]:
!pip install openai

In [ ]:
import os
#Inserindo a chave de API OPENAI_API_KEYcomo variável de ambiente
os.environ['OPENAI_API_KEY'] = 'Digite sua chave de API da OPEN IA Aqui'

In [49]:
#Interagindo com a API da OPEN IA e enviando a transcrição de do áudio para o LLM
from openai import OpenAI

client = OpenAI(api_key=os.environ['OPENAI_API_KEY'],)

response = client.responses.create(
    model="gpt-5-nano",
    input=[{"role":"developer", "content": "Assuma o papel de um Tradutor especialista, que ajuda o usuário a traduzir frases, expressões e palavras para o inglês americano. Quando a tradução for de uma expressão ou palavra, dê pelo menos 2 exemplos de aplicações dessas expressões ou palavra em frases do cotidiano em inglês."},
     {"role": "user", "content":transcription }]
)

print(response.output_text)
response_text = response.output_text

Tradução sugerida (inglês americano): 
- Can you take care of this child?
Alternativas comuns: 
- Could you take care of this child?
- Can you look after this child? 
- Could you watch this child? 
- Could you babysit this child?

Exemplos de uso em frases do cotidiano (em inglês):
- Can you take care of this child for the next two hours while I finish this task?
- She takes care of her younger brother after school.
- Could you look after this child while I make a quick phone call?
- Would you be able to watch this child for a couple of hours until the parents get home?


# 4. Sintetizando a Resposta do ChatGPT Como Voz (gTTS) 🔊

In [50]:
#interagindo com o Text to speech da OpenIA

speech_file_path = "/content/speech.mp3"

with client.audio.speech.with_streaming_response.create(
    model="gpt-4o-mini-tts",
    voice="marin",
    speed = 0.9,
    input= response_text, #response_text é o texto de resposta do LLM, que será transformado em áudio para o usuário
    instructions="Speak in a cheerful and positive tone.",
) as response:
    response.stream_to_file(speech_file_path)

# Reproduz o áudio da resposta salvo no arquivo
display(Audio(speech_file_path, autoplay=True))